In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import numpy as np


import tensorflow as tf
import os

model = load_model("../h5/ft1_mango_leaf_disease.h5")  # or your SavedModel path


In [55]:
data_dir = r"C:\Users\Jaero\Desktop\DP1\Group_2_Repo\Plant_Disease_Prediction\data"
test_dir = os.path.join(data_dir, "test")

img_size = (224, 224)   # MobileNetV2 input size
batch_size = 32

test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=img_size,
    batch_size=batch_size,
)

def preprocess(images, labels):
    images = preprocess_input(images)  # Important for MobileNetV2
    return images, labels

class_names = test_ds.class_names
test_ds = test_ds.map(preprocess)

Found 148 files belonging to 3 classes.


In [59]:
pred_probs = model.predict(test_ds)              # probabilities
predicted_classes = np.argmax(pred_probs, axis=1)

5/5 [==============================] - 2s 170ms/step


In [60]:
loss, acc = model.evaluate(test_ds)
print(f"Test accuracy: {acc:.4f}")
print(f"Test loss: {loss:.4f}")

5/5 [==============================] - 2s 85ms/step - loss: 0.2064 - accuracy: 0.9189
Test accuracy: 0.9189
Test loss: 0.2064


In [61]:
y_true = []
y_pred_classes = []

for images, labels in test_ds:
    preds = model.predict(images)
    y_pred_classes.extend(np.argmax(preds, axis=1))
    y_true.extend(labels.numpy())

y_true = np.array(y_true)
y_pred_classes = np.array(y_pred_classes)

from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true, y_pred_classes, target_names=class_names))

1/1 [==============================] - 1s 991ms/step
              precision    recall  f1-score   support

 Anthracnose       0.98      0.90      0.94        50
     Healthy       0.87      0.96      0.91        50
OtherDisease       0.91      0.90      0.91        48

    accuracy                           0.92       148
   macro avg       0.92      0.92      0.92       148
weighted avg       0.92      0.92      0.92       148



In [28]:
file_paths = test_ds.file_paths

for path, pred, probs in zip(file_paths, predicted_classes, pred_probs):
    predicted_label = class_names[pred]
    confidence = np.max(probs) * 100
    print(f"{path} → {predicted_label} ({confidence:.2f}%)")

C:\Users\Jaero\Desktop\DP1\Group_2_Repo\Plant_Disease_Prediction\data\test\OtherDisease\IMG_20211108_123113 (Custom).jpg → Anthracnose (87.94%)
C:\Users\Jaero\Desktop\DP1\Group_2_Repo\Plant_Disease_Prediction\data\test\Anthracnose\20211008_125104 (Custom).jpg → OtherDisease (89.25%)
C:\Users\Jaero\Desktop\DP1\Group_2_Repo\Plant_Disease_Prediction\data\test\OtherDisease\20211011_162131 (Custom).jpg → Anthracnose (51.76%)
C:\Users\Jaero\Desktop\DP1\Group_2_Repo\Plant_Disease_Prediction\data\test\OtherDisease\20211129_162827 (Custom).jpg → OtherDisease (49.98%)
C:\Users\Jaero\Desktop\DP1\Group_2_Repo\Plant_Disease_Prediction\data\test\Anthracnose\20211011_134909 (Custom).jpg → Anthracnose (90.86%)
C:\Users\Jaero\Desktop\DP1\Group_2_Repo\Plant_Disease_Prediction\data\test\OtherDisease\IMG_20211106_161733 (Custom).jpg → Anthracnose (51.55%)
C:\Users\Jaero\Desktop\DP1\Group_2_Repo\Plant_Disease_Prediction\data\test\Healthy\20211231_160229 (Custom).jpg → OtherDisease (71.84%)
C:\Users\Jaero\D